# Trabalho de Visão Computacional

## Importação das Bibliotecas:

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import glob as gb
import os
from importlib import reload
import sys
from tqdm import tqdm
import pandas as pd
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from PIL import Image

from models.alexNet import MY_AlexNet
from models.vgg16 import MY_VGG16
from utils.utils import *

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


--------------------------------------

## Configuração dos Datasets:

* CIFAR-10:

In [2]:
training_data_C10 = datasets.CIFAR10(
    root="../data/datasets/",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data_C10 = datasets.CIFAR10(
    root="../data/datasets/",
    train=False,
    download=True,
    transform=ToTensor(),
)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
training_data_C10, test_data_C10

(Dataset CIFAR10
     Number of datapoints: 50000
     Root location: ../data/datasets/
     Split: Train
     StandardTransform
 Transform: ToTensor(),
 Dataset CIFAR10
     Number of datapoints: 10000
     Root location: ../data/datasets/
     Split: Test
     StandardTransform
 Transform: ToTensor())

* Building-vs-forests:

In [ ]:
from zipfile import ZipFile

with ZipFile("../data/datasets/buildings-vs-forests/test_set.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/datasets/buildings-vs-forests/")
    
with ZipFile("../data/datasets/buildings-vs-forests/traning_set.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/datasets/buildings-vs-forests/")

In [4]:
tfm = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])   
])

In [5]:
trian_path = '../data/datasets/buildings-vs-forests/traning_set'
test_path = '../data/datasets/buildings-vs-forests/test_set'

training_data_BF = ImageFolder(trian_path, transform = tfm)
test_data_BF = ImageFolder(test_path, transform = tfm)

len_train = len(training_data_BF)
len_test = len(test_data_BF)

training_data_BF.class_to_idx
test_data_BF.class_to_idx

{'buildings': 0, 'forest': 1}

-----------------------

## Treinamento e Avaliação - CIFAR10:

### AlexNet:

* Variando Epócas:

In [3]:
epochs = [5, 10, 20, 50]
accuracys = []
errors = []

accuracy = 0
error = 999

for epoch in tqdm(epochs):
    batch_size = 64
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    for t in range(epoch):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test_for_class(test_dataloader, model, loss_fn, device)
    
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'5 Epochs': accuracys[0], '10 Epochs': accuracys[1], '20 Epochs': accuracys[2], '50 Epochs': accuracys[3]})
df.to_csv('../data/results/cifar10_alexNet_epochs.csv', index=False)
df

100%|██████████| 4/4 [22:02<00:00, 330.74s/it]


,5 Epochs,10 Epochs,20 Epochs,50 Epochs
0,1.0,0.0,0.132,0.140
1,0.0,1.0,0.000,0.304
2,0.0,0.0,0.000,0.169
3,0.0,0.0,0.000,0.000
4,0.0,0.0,0.293,0.002
5,0.0,0.0,0.104,0.001
6,0.0,0.0,0.000,0.000
7,0.0,0.0,0.000,0.000
8,0.0,0.0,0.000,0.220
9,0.0,0.0,0.934,0.783


* Variando Batch Size:

In [6]:
batch_sizes = [64, 128, 256]
accuracys = []
errors = []

accuracy = 0
error = 999

for batch_size in tqdm(batch_sizes):
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test_for_class(test_dataloader, model, loss_fn, device)
        
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Batch Size: 64': accuracys[0], 'Batch Size: 128': accuracys[1], 'Batch Size: 256': accuracys[2]})
df.to_csv('../data/results/cifar10_alexNet_batchSize.csv', index=False)
df

100%|██████████| 3/3 [31:12<00:00, 624.29s/it]


,Batch Size: 64,Batch Size: 128,Batch Size: 256
0,0.796,0.000,0.000
1,0.000,0.000,0.000
2,0.002,0.000,0.000
3,0.000,0.000,0.000
4,0.039,0.009,0.000
5,0.000,0.000,0.002
6,0.000,0.000,0.000
7,0.000,0.000,0.000
8,0.005,0.562,1.000
9,0.529,0.731,0.000


* Variando Taxa de Aprendizado:

In [8]:
learnings = [1e-2, 1e-3, 1e-4]
accuracys = []
errors = []

accuracy = 0
error = 999

for rate in tqdm(learnings):
    train_dataloader = DataLoader(training_data_C10, batch_size=64)
    test_dataloader = DataLoader(test_data_C10, batch_size=64)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=rate)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test_for_class(test_dataloader, model, loss_fn, device)

    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Learning Rate: 1e-2': accuracys[0], 'Learning Rate: 1e-3': accuracys[1], 'Learning Rate: 1e-4': accuracys[2]})
df.to_csv('../data/results/cifar10_alexNet_learnRate.csv', index=False)
df

100%|██████████| 3/3 [38:12<00:00, 764.01s/it]


,Learning Rate: 1e-2,Learning Rate: 1e-3,Learning Rate: 1e-4
0,0.706,0.978,0.009
1,0.785,0.000,0.000
2,0.559,0.016,0.000
3,0.622,0.000,0.000
4,0.549,0.000,0.000
5,0.577,0.000,0.000
6,0.767,0.400,0.999
7,0.684,0.000,0.000
8,0.840,0.002,0.000
9,0.732,0.000,0.000


* Avaliando Melhor Configuração:

In [9]:
accuracys = []
errors = []

accuracy = 0
error = 999

batch_size = 64
epochs = 50
learning = 1e-2

for i in tqdm(range(3)):
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test_for_class(test_dataloader, model, loss_fn, device)

    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Iter 1': accuracys[0], 'Iter 2': accuracys[1], 'Iter 3': accuracys[2]})
df.to_csv('../data/results/cifar10_alexNet_bestConfig.csv', index=False)
df

100%|██████████| 3/3 [39:48<00:00, 796.01s/it]


,Iter 1,Iter 2,Iter 3
0,0.679,0.724,0.679
1,0.761,0.803,0.799
2,0.589,0.367,0.523
3,0.406,0.331,0.627
4,0.756,0.750,0.614
5,0.565,0.548,0.450
6,0.701,0.748,0.739
7,0.742,0.815,0.687
8,0.747,0.690,0.848
9,0.767,0.714,0.800


### VGG16:

* Variando Épocas:

In [10]:
epochs = [5, 10, 20, 50]
accuracys = []
errors = []

accuracy = []
error = 999

for epoch in tqdm(epochs):
    batch_size = 64
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    for t in range(epoch):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test_for_class(test_dataloader, model, loss_fn, device)
    
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'5 Epochs': accuracys[0], '10 Epochs': accuracys[1], '20 Epochs': accuracys[2], '50 Epochs': accuracys[3]})
df.to_csv('../data/results/cifar10_vgg16_epochs.csv', index=False)
df

100%|██████████| 4/4 [1:05:47<00:00, 986.94s/it] 


,5 Epochs,10 Epochs,20 Epochs,50 Epochs
0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,1.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0


* Variando Batch Size:

In [11]:
batch_sizes = [64, 128, 256]
accuracys = []
errors = []

accuracy = 0
error = 999

for batch_size in tqdm(batch_sizes):
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test_for_class(test_dataloader, model, loss_fn, device)
        
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Batch Size: 64': accuracys[0], 'Batch Size: 128': accuracys[1], 'Batch Size: 256': accuracys[2]})
df.to_csv('../data/results/cifar10_vgg16_batchSize.csv', index=False)
df

  0%|          | 0/3 [24:07<?, ?it/s]


KeyboardInterrupt: 

* Variando Taxa de Aprendizado:

In [ ]:
learnings = [1e-2, 1e-3, 1e-4]
accuracys = []
errors = []

accuracy = 0
error = 999

for rate in tqdm(learnings):
    train_dataloader = DataLoader(training_data_C10, batch_size=64)
    test_dataloader = DataLoader(test_data_C10, batch_size=64)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=rate)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test_for_class(test_dataloader, model, loss_fn, device)

    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Learning Rate: 1e-2': accuracys[0], 'Learning Rate: 1e-3': accuracys[1], 'Learning Rate: 1e-4': accuracys[2]})
df.to_csv('../data/results/cifar10_vgg16_learnRate.csv', index=False)
df

* Avaliando Melhor Configuração:

In [ ]:
accuracys = []
errors = []

accuracy = 0
error = 999

batch_size = 64
epochs = 50
learning = 1e-2

for i in tqdm(range(3)):
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test_for_class(test_dataloader, model, loss_fn, device)

    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Iter 1': accuracys[0], 'Iter 2': accuracys[1], 'Iter 3': accuracys[2]})
df.to_csv('../data/results/cifar10_vgg16_bestConfig.csv', index=False)
df

------------------------

## Treinamento e Avaliação - Building-vs-Forest:

### AlexNet:

* Variando Épocas:

In [ ]:
epochs = [5, 10, 20, 50]
accuracys = []
errors = []

accuracy = 0
error = 999

for epoch in tqdm(epochs):
    batch_size = 64
    train_dataloader = DataLoader(training_data_BF, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_BF, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 2)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    for t in range(epoch):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)
    
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Epochs': epochs, 'Accuracy': accuracys, 'Error': errors})
df.to_csv('../data/results/BF_alexNet_epochs.csv', index=False)
df

* Variando Batch Size:

In [ ]:
batch_sizes = [64, 128, 256]
accuracys = []
errors = []

accuracy = 0
error = 999

for batch_size in tqdm(batch_sizes):
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)
        
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Batch Size': batch_sizes, 'Accuracy': accuracys, 'Error': errors})
df.to_csv('../data/results/BF_alexNet_batchSize.csv', index=False)
df

* Variando Taxa de Aprendizado:

In [ ]:
learnings = [1e-2, 1e-3, 1e-4]
accuracys = []
errors = []

accuracy = 0
error = 999

for rate in tqdm(learnings):
    train_dataloader = DataLoader(training_data_C10, batch_size=64)
    test_dataloader = DataLoader(test_data_C10, batch_size=64)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=rate)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)

    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Learning Rate': learnings, 'Accuracy': accuracys, 'Error': errors})
df.to_csv('../data/results/BF_alexNet_learnRate.csv', index=False)
df

* Avaliando Melhor Configuração:

In [ ]:
accuracys = []
errors = []

accuracy = 0
error = 999

batch_size = 64
epochs = 50
learning = 1e-2

for i in tqdm(range(3)):
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)

    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Accuracy': accuracys, 'Error': errors})
df.to_csv('../data/results/BF_alexNet_bestConfig.csv', index=False)
df

### VGG16:

* Variando Épocas:

In [ ]:
epochs = [5, 10, 20, 50]
accuracys = []
errors = []

accuracy = 0
error = 999

for epoch in tqdm(epochs):
    batch_size = 64
    train_dataloader = DataLoader(training_data_BF, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_BF, batch_size=batch_size)

    model = MY_VGG16(n_classes = 2)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    for t in range(epoch):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)
    
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Epochs': epochs, 'Accuracy': accuracys, 'Error': errors})
df.to_csv('../data/results/BF_vgg16_epochs.csv', index=False)
df

* Variando Batch Size:

In [ ]:
batch_sizes = [64, 128, 256]
accuracys = []
errors = []

accuracy = 0
error = 999

for batch_size in tqdm(batch_sizes):
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)
        
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Batch Size': batch_sizes, 'Accuracy': accuracys, 'Error': errors})
df.to_csv('../data/results/BF_vgg16_batchSize.csv', index=False)
df

* Variando Taxa de Aprendizado:

In [ ]:
learnings = [1e-2, 1e-3, 1e-4]
accuracys = []
errors = []

accuracy = 0
error = 999

for rate in tqdm(learnings):
    train_dataloader = DataLoader(training_data_C10, batch_size=64)
    test_dataloader = DataLoader(test_data_C10, batch_size=64)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=rate)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)

    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Learning Rate': learnings, 'Accuracy': accuracys, 'Error': errors})
df.to_csv('../data/results/BF_vgg16_learnRate.csv', index=False)
df

* Avaliando Melhor Configuração:

In [ ]:
accuracys = []
errors = []

accuracy = 0
error = 999

batch_size = 64
epochs = 50
learning = 1e-2

for i in tqdm(range(3)):
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)

    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Accuracy': accuracys, 'Error': errors})
df.to_csv('../data/results/BF_vgg16_bestConfig.csv', index=False)
df

-------------------------------